In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import *
from  tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.python import control_flow_ops

In [ ]:
def CropAndMerge(Input1, Input2, name="bridge"):
        """
        Crop input1 so that it matches input2 and then
        return the concatenation of both channels.
        """
        Size1_x = (Input1).shape[1]
        Size2_x = (Input2).shape[1]

        Size1_y = (Input1).shape[2]
        Size2_y = (Input2).shape[2]
   
        diff_x = tf.divide(tf.subtract(Size1_x, Size2_x), 2)
        diff_y = tf.divide(tf.subtract(Size1_y, Size2_y), 2)
        diff_x = tf.cast(diff_x, tf.int32)
        Size2_x = tf.cast(Size2_x, tf.int32)
        diff_y = tf.cast(diff_y, tf.int32)
        Size2_y = tf.cast(Size2_y, tf.int32)
        crop = tf.slice(Input1, [0, diff_x, diff_y, 0], [-1, Size2_x, Size2_y, -1])
        concat = tf.concat([crop, Input2], axis=3)

        return concat


def bn_conv_relu(input, filters):
    
    x = Conv2D(filters,kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same',use_bias=True, kernel_initializer='glorot_normal',bias_initializer=Constant(0.1))(input)
    x = BatchNormalization(epsilon=1e-3,beta_initializer=Constant(0.0),gamma_initializer=Constant(1.0),momentum=0.5)(x)
    x = Conv2D(filters,kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same',use_bias=True, kernel_initializer='glorot_normal',bias_initializer=Constant(0.1))(x)
    x = BatchNormalization(epsilon=1e-3,beta_initializer=Constant(0.0),gamma_initializer=Constant(1.0),momentum=0.5)(x)
    
    return x

def bn_upconv_relu(input, filters,conc):

    x = Conv2DTranspose(filters=filters, kernel_size=(2, 2),activation='relu', strides=(2, 2), padding='same',kernel_initializer='glorot_normal',bias_initializer=Constant(0.1),use_bias=True)(input)
    x = CropAndMerge(Input1=x,Input2=conc,name='bridge')
    
    return x

def dist(
    
    num_classes=1,
    output_activation='sigmoid'):

    inputs = Input((512,512,3))   
    
    filters = [32,64,128,256,512]
 

    # for l in range(num_layers):
    x_conv1 = bn_conv_relu(inputs, filters[0])
    x_pool1 = MaxPooling2D((2, 2), strides=(2, 2),padding="same")(x_conv1)
    x_conv2 = bn_conv_relu(x_pool1, filters[1])
    x_pool2 = MaxPooling2D((2, 2), strides=(2, 2),padding="same")(x_conv2)  
    x_conv3 = bn_conv_relu(x_pool2, filters[2])
    x_pool3 = MaxPooling2D((2, 2), strides=(2, 2),padding="same")(x_conv3)
    x_conv4 = bn_conv_relu(x_pool3, filters[3])
    x_pool4 = MaxPooling2D((2, 2), strides=(2, 2),padding="same")(x_conv4)
    x_conv5 = bn_conv_relu(x_pool4, filters[4])

# upsampling in the form of convtranspose

    x_tconv5 = bn_upconv_relu(x_conv5, filters[3],x_conv4)
    u_conv4 = bn_conv_relu(x_tconv5, filters[3])
    x_tconv4 = bn_upconv_relu(u_conv4, filters[2],x_conv3)
    u_conv3 = bn_conv_relu(x_tconv4, filters[2])
    x_tconv3 = bn_upconv_relu(u_conv3, filters[1],x_conv2)
    u_conv2 = bn_conv_relu(x_tconv3, filters[1])
    x_tconv2 = bn_upconv_relu(u_conv2, filters[0],x_conv1)
    u_conv1 = bn_conv_relu(x_tconv2, filters[0])
              
    outputs = Conv2D(num_classes, kernel_size=(1,1), strides=(1,1), activation=output_activation, padding='same') (u_conv1)       
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:

model_dist = dist()
adam = keras.optimizers.Adam(lr = 0.001,decay=5*1e-6)
model_dist.compile(optimizer = "adam" , loss = "binary_crossentropy" , metrics = ["acc"])
model_dist.summary()